In [1]:
import pandas as pd
import json
import gc
# import printr
import pickle as pkl

In [2]:
path_data = "/tf/test/WSD/data/keyword_extraction/kor_paper_dataset_KIDA.json"

In [3]:
import ijson
#https://stackoverflow.com/questions/20149321/python-ijson-parse-file-ijson-from-softwaremaniacs-org
#https://github.com/ICRAR/ijson
#https://heegyukim.medium.com/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4-json-%EB%8D%A4%ED%94%84-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%8C%8C%EC%8B%B1%ED%95%98%EA%B8%B0-8f41cee1e155
#https://thewebdev.info/2022/04/17/how-to-load-big-json-files-in-a-memory-efficient-and-fast-way-with-python/

In [37]:
fff = open(path_data)
obj = ijson.items(fff, 'title.item')
for o in obj:
    print(o["ko"])

In [38]:
obj

In [39]:
list(ijson.parse(open(path_data)))

[('', 'start_array', None),
 ('item', 'start_map', None),
 ('item', 'map_key', 'doc_id'),
 ('item.doc_id', 'string', 'CFKO200621252782226'),
 ('item', 'map_key', 'title'),
 ('item.title', 'start_map', None),
 ('item.title', 'map_key', 'ko'),
 ('item.title.ko', 'string', '시간 샘플링 FTIR 분광계의 데이터 처리'),
 ('item.title', 'map_key', 'en'),
 ('item.title.en',
  'string',
  'Data processing for a time-sampled FTIR Spectrometer'),
 ('item.title', 'end_map', None),
 ('item', 'map_key', 'authors'),
 ('item.authors', 'string', '김대성;배효욱;오승일;전형하;박도현'),
 ('item', 'map_key', 'journal'),
 ('item.journal', 'start_map', None),
 ('item.journal', 'map_key', 'ko'),
 ('item.journal.ko', 'string', '한국광학회:학술대회논문집'),
 ('item.journal', 'map_key', 'en'),
 ('item.journal.en',
  'string',
  'Proceedings of the Optical Society of Korea Conference'),
 ('item.journal', 'end_map', None),
 ('item', 'map_key', 'year'),
 ('item.year', 'string', '2006'),
 ('item', 'map_key', 'abstract'),
 ('item.abstract', 'start_map', None),

In [4]:
extract_data_points = [("item.doc_id", "string"), ("item.title.ko", "string"), ("item.abstract.ko", "string"), ("item.abstract.en", "string"), ("item.keywords.ko", "string"), ("item.keywords.en", "string")] #, ("item.body_text.item.text.item", "string")]

In [5]:

def kida_preprocess(nums=-1):
    i=0 
    converted_dict = {}
    text_sum = []
    with open(path_data, encoding='utf-8') as k_file:
        parser = ijson.parse(k_file)
    
        for prefix, event, value in parser:

            # text에 대해서는 여러개 존재가능... 쌓거나 list로 넣도록
            if prefix == "item.body_text.item.text.item":
                text_sum.append(value)
                
            if (prefix, event) in extract_data_points:
                converted_dict[prefix[5:]] = value
        
            if (prefix, event, value) == ("item", "end_map", None):
                converted_dict["body_text.item.text.item"] = text_sum
                # keyword가 none이면 discard
                # 그외 다른것도 none이면 discard
                # 위에 설정한 모든 항목들이 다 들어가 있는 것만 추출 (과연 그럴 필요가 있을지는 전체 데이터 개수 보고 판단.)
                if 'keywords.ko' not in list(converted_dict.keys()) or len(list(converted_dict.keys()))!=7:
                    converted_dict = {}
                    text_sum = []
                    continue
                
                
                yield converted_dict
                converted_dict = {}
                text_sum = []
                i+=1
            
                if nums > 0 and i >=nums:
                    break
            

In [6]:
D_id = []
title = []
abstract_ko = []
abstract_en = []
keywords_ko = []
keywords_en = []
text = []

In [7]:
file_count = 1
for idx, doc in enumerate(kida_preprocess(-1)):
    #print(idx)
    D_id.append(doc['doc_id'])
    title.append(doc['title.ko'])
    abstract_ko.append(doc['abstract.ko'])
    abstract_en.append(doc['abstract.en'])
    key_ko = doc['keywords.ko'].split(';')
    keywords_ko.append(key_ko[:-1])
    key_en = doc['keywords.en'].split(';')
    keywords_en.append(key_en[:-1])
    text.append(doc['body_text.item.text.item'])
    
    if idx != 0 and len(D_id)%1000 == 0:
        print(idx+1)
        kida_data_tbl = pd.DataFrame({'ID': D_id, 'TITLE': title, 'KR_ABSTRACT': abstract_ko, 'EN_ABSTRACT': abstract_en, 'KR_KEYWORDS': keywords_ko, 'EN_KEYWORDS': keywords_en, 'TEXT': text})
        kida_data_tbl.to_pickle("/tf/test/WSD/data/keyword_extraction/preprocessed_pkl/{0:03d}_KIDA_data_tbl.pkl".format(file_count), protocol=4)
        print("File saved!")
        file_count += 1
        D_id = []
        title = []
        abstract_ko = []
        abstract_en = []
        keywords_ko = []
        keywords_en = []
        text = []


1000
File saved!
2000
File saved!
3000
File saved!
4000
File saved!
5000
File saved!
6000
File saved!
7000
File saved!
8000
File saved!
9000
File saved!
10000
File saved!
11000
File saved!
12000
File saved!
13000
File saved!
14000
File saved!
15000
File saved!
16000
File saved!
17000
File saved!
18000
File saved!
19000
File saved!
20000
File saved!
21000
File saved!
22000
File saved!
23000
File saved!
24000
File saved!
25000
File saved!
26000
File saved!
27000
File saved!
28000
File saved!
29000
File saved!
30000
File saved!
31000
File saved!
32000
File saved!
33000
File saved!
34000
File saved!
35000
File saved!
36000
File saved!
37000
File saved!
38000
File saved!
39000
File saved!
40000
File saved!
41000
File saved!
42000
File saved!
43000
File saved!
44000
File saved!
45000
File saved!
46000
File saved!
47000
File saved!
48000
File saved!
49000
File saved!
50000
File saved!
51000
File saved!
52000
File saved!
53000
File saved!
54000
File saved!
55000
File saved!
56000
File saved!
5

In [8]:
check = pkl.load(open("/tf/test/WSD/data/keyword_extraction/preprocessed_pkl/001_KIDA_data_tbl.pkl", 'rb'))

In [9]:
check

,ID,TITLE,KR_ABSTRACT,EN_ABSTRACT,KR_KEYWORDS,EN_KEYWORDS,TEXT
0,JAKO200519663059730,HDR-WPAN 시스템을 위한 주파수 옵셋 보상과 트래킹 알고리즘 성능분석,본 논문에서는 HDR-WPAN 시스템에 적합한 주파수 옵셋 보상과 누적 위상오차를 ...,"In this paper, we proposed the frequency offse...","[홈 네트워크, 주파수 동기, 주파수 옵셋 보상, 트랙킹 알고리즘]",[HDR-WPAN],"[음성, 영상, 데이터 등의 다양한 정보를 실시간으로 전송하기 위해 수십, 수백 M..."
1,JAKO200522656041952,경관지수와 생태계용역가치를 활용한 대구광역도시권 경관의 구조적·기능적 변화 분석,"생태계는 인간, 생물 무생물적 환경의 집합체이며, 경관은 생태계가 단위지역에 나타나...","Ecosystem is composed of human, biotic and abi...","[경관메트릭스, 생태계용역가치, 파편화, 생물대]","[Landscape Metrics, Ecosystem Service Value, F...","[생태계는 인간, 생물체 그리고 무생물적 환경의 총체적 조합(Klijn, 1994)..."
2,JAKO200522656041802,실시간 어장정보 생산 부이시스템 개발 및 활용연구,동해중북부 연안역의 해황 변화에 따른 연안 양식 환경변화에 능동적으로 대처할 수 있...,To provide observed oceanography data at coast...,"[어장정보, 원격탐사, 해황변동, 실시간 모니터링]","[Oceanography Infromation for Fisheries, Remot...",[우리나라 동해중북부 연안역은 북한한류수와 동한난류수가 만나 계절에 따라 세기는 다...
3,JAKO200522656041654,건물분포를 고려한 도시형태의 프랙털(Fractal) 해석,본 연구의 목적은 프랙털(fractal)이론을 이용하여 도시공간을 구성하는 요소들의...,The purpose of this paper is to conduct an exp...,"[도시형태, 프랙털, 프랙털차원, 자기유사성, 복잡함]","[Urban Shape, Fractal, Fractal Dimension, Self...",[현재에 관찰되는 도시의 공간구조는 그 공간 내에서 부분적으로 이루어진 작은 변화가...
4,JAKO200522656041565,경관메트릭스를 이용한 금호강 유역 산림경관의 시·공간적 변화탐지,본 연구의 목적은 각종 환경적 교란이 야기되고 있는 금호강 유역의 경관구조를 정량화...,The purpose of this study is to test the appli...,"[산림경관, 경관메트릭스, 경관생태학, 유역관리, 토지피복]","[Forest Landscape, Landscape Metrics, Landscap...",[Scientists and environmental managers alike a...
...,...,...,...,...,...,...,...
995,JAKO200225459314061,터널 심발부의 선균열을 이용한 발파공법의 현장시험에 관한 연구,터널굴착 발파기술로 새로 개발된 Composition Presplitting Ang...,Composition Presplitting Angle Cut (COPA-Cut) ...,"[심발부, 선균열, COPA - Cut 발파공법]","[Cut, Presplitting, COPA - Cut blasting method]","[우리나라에서 발파공사가 시작 된지는 약 100년이 지났으나, 대형 천공장비에 의한..."
996,JAKO200225459314060,인공신경망을 이용한 터널구간의 암반분류 예측,터널을 설계함에 있어서 굴착방법이나 지보패턴을 결정할 때 어려움을 겪는 주된 요인은...,Most of the problems in dealing with the tunne...,"[인공신경망, 오류역전파 학습알고리즘, 학습된 패턴, 자료의 일치성, RMR 암반분류]","[Artificial neural network (ANN), error back p...",[대심도 산악터널 설계시에 대부분의 현장기술자들은 설계에 필요한 암반에 대한 정보를...
997,JAKO200225459313995,불연속면이 존재하는 지반에서 터널굴착에 의한 하중전이,본 연구에서는 지반내에 뚜렷한 불연속면의 존재로 인한 하중의 비대칭 전이특성과 이완...,"In this study, the influence of the presence o...","[하중전이, 이완영역, 불연속면, 가동판, 바닥판, 토층비, 토압비]","[Load transfer, Loosening zone, Discontinuity ...",[터널의 설계 방법에는 이완하중의 개념에 의거하여 터널에 작용하는 하중을 추정해 이...
998,JAKO200128066987790,오이의 평난지 억제재배를 위한 고랭지육묘 효과,오이의 평난지 억제재배를 위한 고랭지에서의 육묘효과를 구명하고자 본 실험을 수행하였...,This study was conducted to investigate the ef...,"[엽록소, 함량, 온도, T/R율]","[chllorophyll contents, temperature, T/R ratio]","[오이의 성 발현에는 품종 등의 유전적 요인과 온도, 일장, 질소수준, 수분 등의 ..."


## Deprecated

In [61]:
N = 500
with open(path_data, encoding='utf-8') as f:
    head = [next(f) for x in range(N)]
print(len(head))
for l in head:
    print(l)

500
[{

  "doc_id": "CFKO200621252782226",

  "title": {

    "ko": "시간 샘플링 FTIR 분광계의 데이터 처리",

    "en": "Data processing for a time-sampled FTIR Spectrometer"

  },

  "authors": "김대성;배효욱;오승일;전형하;박도현",

  "journal": {

    "ko": "한국광학회:학술대회논문집",

    "en": "Proceedings of the Optical Society of Korea Conference"

  },

  "year": "2006",

  "abstract": {},

  "keywords": {},

  "body_text": [

    {

      "text": [

        "수동형 FTIR(Fourier Transform Infrared) 분광계는 주변 환경의 적외선 복사를 이용해 대기 중의 오염원을 감지하고 식별한다. 분광계는 대기로부터 입사된 적외선 신호를 세기 변조시켜 검출하는 간섭계와 감지된 아날로그 형태의 간섭무늬를 증폭하고 필터링을 거쳐 디지털 신호로 변환해주는 신호처리부, 그리고 가공 가능한 형태의 디지털 데이터를 스펙트럼으로 변환하고 분석하는 운용 소프트웨어로 나눌 수 있다. 간섭계는 분광센서의 핵심적인 기능을 수행하는 조립체로서, 간섭계의 한 쪽 거울을 일정한 광학 경로차를 내도록 구동시킴으로써 입사하는 시준광 (collimated beam)의 세기를 변조시킨다. 이 때 나타나는 변조 광의 세기(간섭무늬, interferogram)는 적외선 검출기에 의해 감지된다. 검출기에 감지된 적외선 신호를 일정한 광 경로 차(Optical Path Difference)로 샘플링하기 위해서는 적외선과 같은 경로로 이동하는 참조 광원(레이저 신호)이 사용된다. 참조 광원으로 사용되는 레이저는 신호처리부에서 증폭기와 비교기를 거쳐 적외선 샘플링 신호로 사용되는데, 참조 광원

In [16]:
with open(path_data, encoding='utf-8') as ff:
    for i, line in enumerate(ff):
        print(line)
        data = json.loads(line)
        if i == 100:
            break

[{



JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 3)

In [5]:
def yield_json(path):
    with open(path, 'r', encoding='utf-8-sig') as f:
        for line in f:
            data = json.loads(line)
            yield data["doc_id"], data["title"], data["abstract"], data["keywords"], data["body_text"]

In [6]:
data = yield_json(path_data)

In [7]:
df = pd.DataFrame.from_records(list(data), columns = ["doc_id", "title", "abstract", "keywords", "body_text"])

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 3)